In [1]:
import findspark, datetime, pytz
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import lit,row_number,col,count

findspark.add_packages('mysql:mysql-connector-java:8.0.11')

In [2]:
agora = datetime.datetime.now(pytz.timezone('America/Sao_paulo'))
strBronzePath = f'../BRONZE/loja_simulada-vendas'
# /home/cj/lisarbjc/lab/lake/users/root/CASE-ingestao-gold.ipynb
# [PATH_NOT_FOUND] Path does not exist: file:/home/cj/lisarbjc/lab/lake/users/root/BRONZE/loja_simulada-vendas-clientes.

In [3]:
import os 
dir_path = os.getcwd()
print(dir_path)

/home/cj/lisarbjc/lab/lake/root


In [4]:
# Create SparkSession
spark = (SparkSession.builder
           .appName('IngestaoBronze')
           .config("packages", "org.apache.spark:mysql-connector-java-8.0.13.jar")
           .getOrCreate()
        )

24/06/14 23:26:51 WARN Utils: Your hostname, cj resolves to a loopback address: 127.0.1.1; using 192.168.15.34 instead (on interface enp2s0)
24/06/14 23:26:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/cj/.local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/cj/.ivy2/cache
The jars for the packages stored in: /home/cj/.ivy2/jars
mysql#mysql-connector-java added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-97f65ce2-836c-4f7c-9500-c7f6d0183016;1.0
	confs: [default]
	found mysql#mysql-connector-java;8.0.11 in central
	found com.google.protobuf#protobuf-java;2.6.0 in central
downloading https://repo1.maven.org/maven2/mysql/mysql-connector-java/8.0.11/mysql-connector-java-8.0.11.jar ...
	[SUCCESSFUL ] mysql#mysql-connector-java;8.0.11!mysql-connector-java.jar (961ms)
downloading https://repo1.maven.org/maven2/com/google/protobuf/protobuf-java/2.6.0/protobuf-java-2.6.0.jar ...
	[SUCCESSFUL ] com.google.protobuf#protobuf-java;2.6.0!protobuf-java.jar(bundle) (280ms)
:: resolution report :: resolve 2532ms :: artifacts dl 1254ms
	:: modules in use:
	com.google.protobuf#protobuf-java;2.6.0 from central in [default]
	mysql#mysql-connector-java;8.0.11 from central in [default]
	------

In [5]:
def pegarBronzeAtual(tabela):
    try:
        tempDF = (spark.read.parquet(f'{strBronzePath}-{tabela}')).cache()
        print(f'Base BRONZE "{tabela}" encontrada!')
        return tempDF
    except Exception as e:
        print(f'Base BRONZE "{tabela}" NÃO encontrada!\n', e)
        return None

In [6]:
def enviarAlerta(tipoAlerta, mensagem):
    print(F'Encaminhando alerta de {tipoAlerta}')
    print(f'{tipoAlerta} - type: {type(mensagem)} - Mensagem:', mensagem)
    return

In [7]:
# Read from MySQL Table
def pegarDadosDaTable(tabela):
    try:
        return (spark.read
                .format("jdbc")
                .option("driver","com.mysql.cj.jdbc.Driver")
                .option("url", "jdbc:mysql://localhost:3306/vendas?allowPublicKeyRetrieval=true&useSSL=false")
                .option("dbtable", tabela)
                .option("user", "big_data_importer")
                .option("password", "big_data_importer")
                .load()
             )
    except:
        enviarAlerta('Erro', f'Não foi possível se conectar ao MYSQL - tabela "{tabela}"')
        return None

In [8]:
def salvarFinalDF(df,tabela):
    return df.write.mode("overwrite").parquet(f'{strBronzePath}-{tabela}')

In [9]:
def clientes():
    tabela = 'clientes'
    bronzeAtualDF = pegarBronzeAtual(tabela)
    baseNovaTempDF = pegarDadosDaTable(tabela)
    if baseNovaTempDF == None:
        return
    baseNovaDF = baseNovaTempDF.withColumn("data_carga", lit(agora.strftime("%Y-%m-%d")).cast('date'))
    enviarAlerta('Informacao', f'COUNT bronze {tabela}: {baseNovaDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    if bronzeAtualDF != None:
   # SELECIONAR DADOS NOVOS
        apenasNovosDF = (baseNovaDF.join(bronzeAtualDF          
            ,((bronzeAtualDF.id_cliente == baseNovaDF.id_cliente) &
                (bronzeAtualDF.cpf == baseNovaDF.cpf) &
                (bronzeAtualDF.telefone == baseNovaDF.telefone) &
                (bronzeAtualDF.email == baseNovaDF.email) &
                (bronzeAtualDF.cliente == baseNovaDF.cliente) &
                (bronzeAtualDF.estado == baseNovaDF.estado) &
                (bronzeAtualDF.origem_racial == baseNovaDF.origem_racial) &
                (bronzeAtualDF.sexo == baseNovaDF.sexo) &
                (bronzeAtualDF.status == baseNovaDF.status))
            ,"leftanti"))
        enviarAlerta('Informacao', f'COUNT novos registros: {apenasNovosDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    # ENCONTRAR ULTIMA SURROGATE KEY
        maxSK = bronzeAtualDF.selectExpr("max(surrogate_key) as max_key").collect()[0]["max_key"]
    # ADICIONAR SK AO NOVO DF
        windowSpec = Window.orderBy("id_cliente")        
        apenasNovosSKDF = (apenasNovosDF.withColumn("surrogate_key", maxSK + row_number().over(windowSpec)))
    # FINAL DO PROCESSO
        finalDF = bronzeAtualDF.unionByName(apenasNovosSKDF)
        salvarFinalDF(finalDF, tabela)
        enviarAlerta('Informacao',
            f'Registros a serem salvos: {apenasNovosDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    else:
        windowSpec = Window.orderBy("id_cliente")
        bronzeNovoDF = baseNovaDF.withColumn("surrogate_key", row_number().over(windowSpec))
        salvarFinalDF(bronzeNovoDF, tabela)
        enviarAlerta('Informacao',
            f'Registros a serem salvos: {bronzeNovoDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    return

In [10]:
def vendedores():
    tabela = 'vendedores'
    bronzeAtualDF = pegarBronzeAtual(tabela)
    baseNovaTempDF = pegarDadosDaTable(tabela)
    if baseNovaTempDF == None:
        return
    baseNovaDF = baseNovaTempDF.withColumn("data_carga", lit(agora.strftime("%Y-%m-%d")).cast('date'))
    enviarAlerta('Informacao', f'COUNT bronze {tabela}: {baseNovaDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    if bronzeAtualDF != None:
    # SELECIONAR DADOS NOVOS
        apenasNovosDF = (baseNovaDF.join(bronzeAtualDF          
            ,((bronzeAtualDF.id_vendedor == baseNovaDF.id_vendedor) &
                (bronzeAtualDF.cpf == baseNovaDF.cpf) &
                (bronzeAtualDF.telefone == baseNovaDF.telefone) &
                (bronzeAtualDF.email == baseNovaDF.email) &
                (bronzeAtualDF.origem_racial == baseNovaDF.origem_racial) &
                (bronzeAtualDF.nome == baseNovaDF.nome) )
            ,"leftanti"))
        enviarAlerta('Informacao', f'COUNT novos registros: {apenasNovosDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    # ENCONTRAR ULTIMA SURROGATE KEY
        maxSK = bronzeAtualDF.selectExpr("max(surrogate_key) as max_key").collect()[0]["max_key"]
    # ADICIONAR SK AO NOVO DF
        windowSpec = Window.orderBy("id_vendedor")        
        apenasNovosSKDF = (apenasNovosDF.withColumn("surrogate_key", maxSK + row_number().over(windowSpec)))
    # FINAL DO PROCESSO
        finalDF = bronzeAtualDF.unionByName(apenasNovosSKDF)
        salvarFinalDF(finalDF, tabela)
        enviarAlerta('Informacao',
            f'Registros a serem salvos: {finalDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    else:
        windowSpec = Window.orderBy("id_vendedor")
        bronzeNovoDF = baseNovaDF.withColumn("surrogate_key", row_number().over(windowSpec))
        salvarFinalDF(bronzeNovoDF, tabela)
        enviarAlerta('Informacao',
            f'Registros a serem salvos: {bronzeNovoDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    return

In [11]:
def produtos():
    tabela = 'produtos'
    bronzeAtualDF = pegarBronzeAtual(tabela)
    baseNovaTempDF = pegarDadosDaTable(tabela)
    if baseNovaTempDF == None:
        return
    baseNovaDF = baseNovaTempDF.withColumn("data_carga", lit(agora.strftime("%Y-%m-%d")).cast('date'))
    enviarAlerta('Informacao', f'COUNT bronze {tabela}: {baseNovaDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    if bronzeAtualDF != None:
    # SELECIONAR DADOS NOVOS
        apenasNovosDF = (baseNovaDF.join(bronzeAtualDF          
            ,((bronzeAtualDF.id_produto == baseNovaDF.id_produto) &
                (bronzeAtualDF.produto == baseNovaDF.produto) &
                (bronzeAtualDF.preco == baseNovaDF.preco))
            ,"leftanti"))
        enviarAlerta('Informacao', f'COUNT novos registros: {apenasNovosDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    # ENCONTRAR ULTIMA SURROGATE KEY
        maxSK = bronzeAtualDF.selectExpr("max(surrogate_key) as max_key").collect()[0]["max_key"]
    # ADICIONAR SK AO NOVO DF
        windowSpec = Window.orderBy("id_produto")        
        apenasNovosSKDF = (apenasNovosDF.withColumn("surrogate_key", maxSK + row_number().over(windowSpec)))
    # FINAL DO PROCESSO
        finalDF = bronzeAtualDF.unionByName(apenasNovosSKDF)
        salvarFinalDF(finalDF, tabela)
        enviarAlerta('Informacao',
            f'Registros a serem salvos: {finalDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    else:
        windowSpec = Window.orderBy("id_produto")
        bronzeNovoDF = baseNovaDF.withColumn("surrogate_key", row_number().over(windowSpec))
        salvarFinalDF(bronzeNovoDF, tabela)
        enviarAlerta('Informacao',
            f'Registros a serem salvos: {bronzeNovoDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    return

In [12]:
def vendas():
    tabela = 'vendas'
    bronzeAtualDF = pegarBronzeAtual(tabela)
    baseNovaTempDF = pegarDadosDaTable(tabela)
    if baseNovaTempDF == None:
        return
    baseNovaDF = baseNovaTempDF.withColumn("data_carga", lit(agora.strftime("%Y-%m-%d")).cast('date'))
    enviarAlerta('Informacao', f'COUNT bronze {tabela}: {baseNovaDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    if bronzeAtualDF != None:
    # SELECIONAR DADOS NOVOS
        apenasNovosDF = (baseNovaDF.join(bronzeAtualDF          
            ,((bronzeAtualDF.id_venda == baseNovaDF.id_venda) &
                (bronzeAtualDF.id_vendedor == baseNovaDF.id_vendedor) &
                (bronzeAtualDF.id_cliente == baseNovaDF.id_cliente) &
                (bronzeAtualDF.data == baseNovaDF.data)) &
                (bronzeAtualDF.total == baseNovaDF.total)
            ,"leftanti"))
        enviarAlerta('Informacao', f'COUNT novos registros: {apenasNovosDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    # ENCONTRAR ULTIMA SURROGATE KEY
        maxSK = bronzeAtualDF.selectExpr("max(surrogate_key) as max_key").collect()[0]["max_key"]
    # ADICIONAR SK AO NOVO DF
        windowSpec = Window.orderBy("id_venda")        
        apenasNovosSKDF = (apenasNovosDF.withColumn("surrogate_key", maxSK + row_number().over(windowSpec)))
    # FINAL DO PROCESSO
        finalDF = bronzeAtualDF.unionByName(apenasNovosSKDF)
        salvarFinalDF(finalDF, tabela)
        enviarAlerta('Informacao',
            f'Registros a serem salvos: {finalDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    else:
        windowSpec = Window.orderBy("id_venda")
        bronzeNovoDF = baseNovaDF.withColumn("surrogate_key", row_number().over(windowSpec))
        salvarFinalDF(bronzeNovoDF, tabela)
        enviarAlerta('Informacao',
            f'Registros a serem salvos: {bronzeNovoDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    return

In [13]:
def itens_venda():
    tabela = 'itens_venda'
    bronzeAtualDF = pegarBronzeAtual(tabela)
    baseNovaTempDF = pegarDadosDaTable(tabela)
    if baseNovaTempDF == None:
        return
    baseNovaDF = baseNovaTempDF.withColumn("data_carga", lit(agora.strftime("%Y-%m-%d")).cast('date'))
    enviarAlerta('Informacao', f'COUNT bronze {tabela}: {baseNovaDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    if bronzeAtualDF != None:
    # SELECIONAR DADOS NOVOS
        # bronzeAtualDF = bronzeAtualDF1  # .select('*').where(col('id_venda') < 10)  # TODO inserida para teste, retirar
        apenasNovosDF = (baseNovaDF.join(bronzeAtualDF          
            ,((bronzeAtualDF.id_produto == baseNovaDF.id_produto) &
                (bronzeAtualDF.id_venda == baseNovaDF.id_venda) &
                (bronzeAtualDF.quantidade == baseNovaDF.quantidade) &
                (bronzeAtualDF.valor_unitario == baseNovaDF.valor_unitario)) &
                (bronzeAtualDF.valor_total == baseNovaDF.valor_total) &
                (bronzeAtualDF.desconto == baseNovaDF.desconto)
            ,"leftanti"))
        enviarAlerta('Informacao', f'COUNT novos registros: {apenasNovosDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    # ENCONTRAR ULTIMA SURROGATE KEY
        maxSK = bronzeAtualDF.selectExpr("max(surrogate_key) as max_key").collect()[0]["max_key"]
    # ADICIONAR SK AO NOVO DF
        windowSpec = Window.orderBy(*["id_venda", "id_produto"])
        apenasNovosSKDF = (apenasNovosDF.withColumn("surrogate_key", maxSK + row_number().over(windowSpec)))
    # FINAL DO PROCESSO
        finalDF = bronzeAtualDF.unionByName(apenasNovosSKDF)
        salvarFinalDF(finalDF, tabela)
        enviarAlerta('Informacao',
            f'Registros a serem salvos: {finalDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    else:
        windowSpec = Window.orderBy("id_venda")
        bronzeNovoDF = baseNovaDF.withColumn("surrogate_key", row_number().over(windowSpec))
        salvarFinalDF(bronzeNovoDF, tabela)
        enviarAlerta('Informacao',
            f'Registros a serem salvos: {bronzeNovoDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    return

In [14]:
def main():
    try:
        enviarAlerta('Informacao', 'ingestão para base BRONZE iniciada...')
        clientes()
        vendedores()
        produtos()
        vendas()
        itens_venda()
        enviarAlerta('Sucesso', 'Ingestão para base BRONZE concluída!!!')
    except Exception as e:
        enviarAlerta('Erro', e)
    finally:
        spark.stop()
        print('Conexão spark finalizada!')

In [15]:
main()

Encaminhando alerta de Informacao
Informacao - type: <class 'str'> - Mensagem: ingestão para base BRONZE iniciada...


Base BRONZE "clientes" encontrada!
Encaminhando alerta de Informacao
Informacao - type: <class 'str'> - Mensagem: COUNT bronze clientes: 250
Encaminhando alerta de Informacao
Informacao - type: <class 'str'> - Mensagem: COUNT novos registros: 0


24/06/14 23:27:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/14 23:27:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/14 23:27:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/14 23:27:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/14 23:27:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/14 23:27:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/14 2

Encaminhando alerta de Informacao
Informacao - type: <class 'str'> - Mensagem: Registros a serem salvos: 0
Base BRONZE "vendedores" encontrada!
Encaminhando alerta de Informacao
Informacao - type: <class 'str'> - Mensagem: COUNT bronze vendedores: 10
Encaminhando alerta de Informacao
Informacao - type: <class 'str'> - Mensagem: COUNT novos registros: 0


24/06/14 23:27:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/14 23:27:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/14 23:27:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/14 23:27:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/14 23:27:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/14 23:27:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


...

Encaminhando alerta de Informacao
Informacao - type: <class 'str'> - Mensagem: Registros a serem salvos: 10
Base BRONZE "vendas" encontrada!
Encaminhando alerta de Informacao
Informacao - type: <class 'str'> - Mensagem: COUNT bronze vendas: 20
Encaminhando alerta de Informacao
Informacao - type: <class 'str'> - Mensagem: COUNT novos registros: 20


24/06/14 23:27:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/14 23:27:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/14 23:27:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/14 23:27:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/14 23:27:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/14 23:27:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/14 2

Encaminhando alerta de Informacao
Informacao - type: <class 'str'> - Mensagem: Registros a serem salvos: 299
Base BRONZE "itens_venda" encontrada!
Encaminhando alerta de Informacao
Informacao - type: <class 'str'> - Mensagem: COUNT bronze itens_venda: 20
Encaminhando alerta de Informacao
Informacao - type: <class 'str'> - Mensagem: COUNT novos registros: 19


24/06/14 23:27:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/14 23:27:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/14 23:27:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/14 23:27:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/14 23:27:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/14 23:27:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/14 2

Encaminhando alerta de Informacao
Informacao - type: <class 'str'> - Mensagem: Registros a serem salvos: 298
Encaminhando alerta de Sucesso
Sucesso - type: <class 'str'> - Mensagem: Ingestão para base BRONZE concluída!!!
Conexão spark finalizada!
